# SHINY APP

Code for the creation of the interactive tool made with Shiny (from RStudios)

In [ ]:
library(ggplot2)
library(shiny)
library(shinyWidgets)
library(dplyr)
library(shinyjs) 
library(tidyverse)
library(data.table)
library(magrittr)
library(shinyhelper)

#Upload the table with all the information
dades <- fread('TaulaExemple_sum.txt',  sep = '\t', dec ='.', header = T) 

llista_rsid <- c('')

#Vector of all populations from the 1000GP
pops <- c('ACB', 'ASW', 'BEB', 'CDX', 'CEU', 'CHB', 'CHS', 'ESN', 
          'FIN', 'GBR', 'GIH', 'GWD', 'IBS', 'ITU', 'JPT', 'KHV', 
          'LWK', 'MSL', 'PJL', 'STU', 'TSI', 'YRI')

#Vector with all the VEP consequences
VEPLabels <- c("transcript_ablation","splice_acceptor_variant","splice_donor_variant","stop_gained",
               "frameshift_variant","stop_lost","start_lost","transcript_amplification",
               "inframe_insertion","inframe_deletion","missense_variant","protein_altering_variant",
               "splice_region_variant","incomplete_terminal_codon_variant","start_retained_variant",
               "stop_retained_variant","synonymous_variant","coding_sequence_variant",
               "mature_miRNA_variant","5_prime_UTR_variant","3_prime_UTR_variant",
               "non_coding_transcript_exon_variant","intron_variant","NMD_transcript_variant",
               "non_coding_transcript_variant","upstream_gene_variant","downstream_gene_variant",
               "TFBS_ablation","TFBS_amplification","TF_binding_site_variant",
               "regulatory_region_ablation","regulatory_region_amplification","feature_elongation",
               "regulatory_region_variant","feature_truncation","intergenic_variant")

#Vector with all the Clinical Significances
ClinVarLabels <- c('Pathogenic', 'Pathogenic/likely pathogenic', 'Likely pathogenic',
                   'drug response', 'association', 'risk factor', 'affects', 'other',
                   'conflicting interpretations of pathogenicity', 
                   'conflicting interpretations from submitters', 'uncertain significance',
                   'no interpretation for the single variant', 'not provided', 
                   'association not found', 'protective', 'Likely benign', 
                   'Benign/Likely benign', 'Benign')

#Vector with all the Regulome Rank Scores
regulomeLabels <- c('1a', '1b', '1c', '1d', '1e', '1f', '2a',
                    '2b', '2c', '3a', '3b', '4','5','6','7')



ui <- fluidPage(

    # Application title
    titlePanel("Functional Description"),
    
    sidebarLayout(
        
        #Sidebar panel (inputs)
        sidebarPanel(
            
            #Filter title
            h4('Location filters'),
            
            #Filters to select region
            selectInput('chrInput', 'Chromosome:',
                        choices = c(1:22),
                        selected = 22),
            
            textInput('minPosInput', 'Min Position', value = min(dades$physicalPos)),
            
            textInput('maxPosInput', 'Max Position', value = max(dades$physicalPos)),


            hr(), br(), #Separators
            
            #Filter title
            h4('Population metrics filters'),
            
            #Filter by extreme values
            checkboxInput('extremeValuesInput', 'Filter Extreme Values for IHS'),
            
            #Filter to select the populations where we apply the extreme values filter
            checkboxGroupButtons('popsInput', 'Populations:', 
                               choices = pops, selected = pops),
            
            #Button to select/deselect all populations (all selected by default)
            actionButton('selectAllPopsInput', 'Select / Deselect All'), 
            
            hr(), br(), #Separators
            
            #Filter title
            h4('Genome-wide Association Studies (GWAS) Catalog filter'),
            
            #Filter by GWAS association counts + help button with info about the filter
            checkboxInput('filterGWASInput','FIlter by GWAS Association Counts' )%>%
            helper(type = "inline", 
                   title = "GWAS Filter Help",
                   content = c("GWAS Association counts filters the genome variants by how many times each variant has been reported in a GWAS study.",
                               "GWAS catalog: <a>https://www.ebi.ac.uk/gwas</a>"),
                   size = "m"),
            
            #Slider to select the range of GWAS associations
            sliderInput('GWASInput', 'GWAS Association Counts:',
                        min = 0, max = max(dades$GWASAssocCounts),
                        value = c(0, max), step = 1), 

            hr(), br(), #Separators
            
            #Filter title
            h4('Variant Effect Predictor (VEP) filter'),
            
            #Filter by VEP consequences + help button with more info about the filter
            checkboxInput('filterVEPInput', 'Filter by VEP Consequences') %>%
                helper(type = "inline",
                       title = "VEP Filter Help",
                       content = c("VEP Consequences filters the genome variants by 
                                   the effect each allele may have on each transcript.",
                                   'The consequence terms are defined by Sequence Ontology',
                                   "The description of each consequence can be found here:", 
                                   "<a>https://m.ensembl.org/info/genome/variation/prediction/predicted_data.html</a>",
                                   'VEP Webpage: <a>https://www.ensembl.org/info/docs/tools/vep/index.html</a>'),
                       size = "m"),
            
            #Checkboxes to select which VEP Consequences filter for
            checkboxGroupInput('VEPInput', 'VEP Consequences',
                                 choices =  VEPLabels),
            
            #Button to select/deselect all
            actionButton('selectAllVEPInput', 'Select / Deselect All'),
            
            hr(), br(), #Separator
            
            #Filter title
            h4('ClinVar filter'),
            
            #Filter by Clinical Significances + help button with more info about the filter
            checkboxInput('filterClinVarInput', 'Filter by Clinical Significances') %>%
                helper(type = "inline",
                       title = "ClinVar Filter Help",
                       content = c("Clinical Significance filters the genome variants by the interpretation of the relationship of a variation to a condition, these interpretations are provided only by submitters.",
                                   'Representation of Clinical Significances can be found here:', 
                                   '<a>https://www.ncbi.nlm.nih.gov/clinvar/docs/clinsig</a>',
                                   "ClinVar Webpage: <a>https://www.ncbi.nlm.nih.gov/clinvar</a>"),
                       size = "m"),
            
            #Checkboxes to select for which Clinical Significances filter
            checkboxGroupInput('ClinVarInput', 'Clinical Significances:',
                               choices = ClinVarLabels),
            
            #Select/deselect all button
            actionButton('selectAllClinVarInput', 'Select / Deselect All'),
            
            hr(), br(),  #Separator
            
            #Filter tilte
            h4('RegulomeDB filter'),
            
            #Filter by RegulomeDB rank Score + help button with more info about the filter
            checkboxInput('filterRegulomeInput', 'Filter by RegulomeDB Rank Score') %>%
                helper(type = "inline",
                       title = "RegulomeDB Filter Help",
                       content = c("Regulome rank filters the genome variants by putative regulatory potential.",
                                   'Representation of RegulomeDB Ranking can be found here: <a>https://regulomedb.org/regulome-help</a>',
                                   "RegulomeDB Webpage: <a>https://regulomedb.org/regulome-search</a>"),
                       size = "m"),
            
            #Slider to select the range of values of Regulome Rank Score 
            sliderTextInput('RegulomeInput', 'Regulome Score:',
                            choices = regulomeLabels, selected = range(regulomeLabels),
                            grid = TRUE),
            
            hr(), br(), #Separator
            
            #Filter title + help button with general info about the database
            h4('DisGeNET filters')%>%
                helper(type = "inline",
                       title = "DisGeNET Filter Help",
                       content = c("Each DisGeNET specific filter is explained below.",
                                   "DisGeNET Webpage: <a>https://www.disgenet.org/home</a>"),
                       size = "m"),
            
            #Filter by DSI + help button with info about the filter
            checkboxInput('filterDSIInput', 'Filter by Disease Specificity Index') %>%
                helper(type = "inline",
                       title = "Disease Specificity Index Filter Help",
                       content = c("Disease Specificity Index (DSI) reflects if a variant (or gene) is associated to several or fewer diseases.",
                                   "The DSI ranges from 0.25 to 1. For example: TNF, associated to more than 1,500 diseases has a DSI of 0.263, while HCN2 is associated to one disease has a DSI of 1.",
                                   "DisGeNet Metrics Info: <a>https://www.disgenet.org/dbinfo</a>"),
                       size = "m"),
            
            #Slider to choose the range of DSI values to filter for
            sliderInput('DSIInput', 'Disease Specificity Index:', 
                         min = 0.25, max = 1, 
                         value = c(0,1)),
            
            #Filter by DPI + help button with info about the filter
            checkboxInput('filterDPIInput', 'Filter by Disease Pleiotropy Index') %>%
            helper(type = "inline",
                   title = "Disease Pleiotropy Index Filter Help",
                   content = c("Disease Pleiotropy Index (DPI) is similar to DSI but it considers if the multiple diseases associated to the variant (or gene) are similar among them or are completely different diseases and belong to different disease clases.",
                               "DisGeNet Metrics Info: <a>https://www.disgenet.org/dbinfo</a>"),
                   size = "m"),
            
            #Slider to choose the range of DPI values to filter for
            sliderInput('DPIInput', 'Disease Pleiotropy Index:', 
                        min = 0, max = 1, 
                        value = c(0,1)),
            
            #Checkbox to filter by DisGeNET score + help button with info about the filter
            checkboxInput('filterScoreInput', 'Filter by Variant-Disease Association Score') %>%
            helper(type = "inline",
                   title = "DisGeNET Score Filter Help",
                   content = c("DisGeNET Score ranks the variant-disease association according to their level of evidence.",
                               "DiSeNet Metrics Info: <a>https://www.disgenet.org/dbinfo</a>"),
                   size = "m"),
            
            #Slider to choose the range of score values to filter for
            sliderInput('DGNscoreInput', 'Variant-Disease Association Score:', 
                        min = 0, max = 1, 
                        value = c(0,1)),
            
            #Filter by N.of Pubmed ids
            checkboxInput('filterNofPmidsInput', 'Filter by Number of Pubmed IDs'),
            
            #Slider to choose Number of Pubmed IDs values to filter for
            sliderInput('NofPmidsInput', 'Number of Pubmed IDs:', 
                        min = 0, max = max(dades$`max(NofPmids)`, na.rm = TRUE), 
                        value = c(0, max)),
           
            hr(), br(),  #Separator  
            
            #Filter title
            h4('Atlas of Variant Age filters'), 
            
            #Filter by age mean and Quality score
            checkboxInput('filterAgeInput', 'Filter by Age Mean and Quality Score'),
            
            #Select by which clock model filter the age mean + help button
            radioButtons('AgeModelInput', 'Atlas of Variant Age Clock Model:',
                         choices = c('Mut', 'Rec', 'Jnt'), 
                         selected = 'Jnt') %>%
                helper(type = "inline",
                       title = "Clock Model Filter Help",
                       content = c("The Genealogical Estimation of Variant Age can be based on three different
                                   coalescent-base clock models (Mutation, Recombination and Joint Clock).",
                                   "More information on the models can be found here: 
                                   <a>https://human.genome.dating/info/about</a>"),
                       size = "m"),
            
            #Slider to choose the range of Age mean value to filter for + help button
            sliderInput('AgeMeanInput', 'Age Mean (years ago):', 
                        min = min(dades$AgeMean_Jnt, na.rm = TRUE),
                        max = max(dades$AgeMean_Jnt, na.rm = TRUE), 
                        value = c(min, max)) %>%
                helper(type = "inline",
                       title = "Age Mean Filter Help",
                       content = c("Age mean filters those genomic variants that have an age mean estimation
                                   of derived allele in the selected range of values.",
                                   "Human Genome Dating webpage: 
                                   <a>https://human.genome.dating/info/about</a>"),
                       size = "m"),
            #Slider to choose the range of score value to filter for
            sliderInput('qualScoreInput', 'Quality Score:',
                        min = 0, max = 1, value = c(0,1)),
            
            hr(), #separator
            
            #Button to download the filtered table
            downloadBttn('downloadInput', 'Download filtered Datatable'),
           
            #Color black the separator line          
            tags$head(
                tags$style(HTML("hr {border-top: 1px solid #000000;}"))
            )
        ),

        
        #Main panel (ouptuts)
        mainPanel(
            
            uiOutput('qresults'), br(), br(), #Textline with number of results found
           
            h3('iHS histogram', align = 'center'), #plot title
            plotOutput('plotIHS'),  #Histogram of iHS values for all populations
            p('iHS values from all populations are represented', align = 'center'), br(), br(), #Plot description
            
            h3('GWAS Association Counts', align = 'center'), #plot title
            plotOutput('plotGWAS'), br(), br(), #Histogram of GWAS Association Counts
            
            h3('VEP Consequences', align = 'center'), #plot title
            plotOutput('plotVEP'), br(), br(), #Histogram of VEP consequences
            
            h3('Clinical Significances from ClinVar', align = 'center'), #plot title
            plotOutput('plotClinVar'), br(), br(), #Histogram of Clinical Significances
            
            h3('RegulomeDB Ranking', align = 'center'), #plot title
            plotOutput('plotRegulome'), br(), br(), #Histogram of RegulomeDB rank
            
            h3('DisGeNET Histograms', align = 'center'), br(), 
            
            h4('Disease Specificity Index'), #plot title
            plotOutput('plotDSI'), br(), br(), #Histogram of DSI
            
            h4('Disease Pleiotropy Index'), #plot title
            plotOutput('plotDPI'), br(), br(), #Histogram of DPI
            
            h4('DisGeNET Score'), #plot title
            plotOutput('plotScore'), br(), br(), #Histogram of Disgenet Score
            
            h4('Number Of PubMed IDs'), #plot title
            plotOutput('plotNofPmids'), br(), br(), #Histogram of Number of PubMed IDs
            
            h3('Atlas of Variant Age Histograms', align = 'center'), br(),
            
            h4('Age Mean by Mutation Clock'), #plot title
            plotOutput('plotAgeMeanMut'), br(), br(), #Histogram of Age Mean by mutation clock model
            
            h4('Age Mean by Recombination Clock'), #plot title
            plotOutput('plotAgeMeanRec'), br(), br(), #Histogram of Age Mean by recombination clock model
            
            h4('Age Mean by Joint Clock'), #plot title
            plotOutput('plotAgeMeanJnt'), br(), br(), #Histogram of Age Mean by joint clock model
            
            h3('Table with the filtered results'), #Table title
            dataTableOutput('tableResults') #Data table with the filtered results
        
        )
    )
)

# Define server logic 
server <- function(input, output, session) {
    
    #Output for help buttons
    observe_helpers(withMathJax = T)
    
    #Select or deselect all populations
    observe({
        if (input$selectAllPopsInput > 0){
            if (input$selectAllPopsInput %% 2 == 0){
                updateCheckboxGroupButtons(session, 'popsInput', selected = pops)
            }
            else {
                updateCheckboxGroupButtons(session, 'popsInput', selected = '')
            }
        }
    })
    
    #Select or deselect all VEP consequences
    observe({
        if (input$selectAllVEPInput > 0){
            if (input$selectAllVEPInput %% 2 == 0){
                updateCheckboxGroupInput(session, 'VEPInput', selected = '')
            }
            else {
                updateCheckboxGroupInput(session, 'VEPInput', selected = VEPLabels)
            }
        }
    })
    
    #Select or deselect all Clinical Significances
    observe({
        if (input$selectAllClinVarInput > 0){
            if (input$selectAllClinVarInput %% 2 == 0){
                updateCheckboxGroupInput(session, 'ClinVarInput', selected = '')
            }
            else {
                updateCheckboxGroupInput(session, 'ClinVarInput', selected = ClinVarLabels)
            }
        }
    })
    
    #Change minimum and maximum values from Age Mean slider depending on the clock model selected
    observeEvent(input$AgeModelInput,{
        updateSliderInput(session, 'AgeMeanInput',
                          min = min(dades[[paste0('AgeMean_',input$AgeModelInput)]], na.rm = TRUE),
                          max = max(dades[[paste0('AgeMean_',input$AgeModelInput)]], na.rm = TRUE),
                          value = c(min,max))
        
    })
   
    
    #TEXT OUTPUT WITH THE NUMBER OF RESULTS FOUND
    output$qresults <- renderUI({
        
        #Filter the original data by position and save it in filtered
        filtered <- dades %>% filter(chr %in% input$chrInput,
                                     physicalPos >= input$minPosInput,
                                     physicalPos <= input$maxPosInput)
        
        #if extreme values checkbox is selected, select those variants that have 
        #an extreme value in at least one of the populations selected
        if (input$extremeValuesInput == TRUE){
            for (POP in pops){
                if (POP %in% input$popsInput){
                    llista_rsid <- c(llista_rsid, 
                                     filtered %>% filter(!is.na(filtered[[paste0('extremevalue_',POP)]])) %>%
                                         select(rsid))
                        
                    }
            }
                filtered <- filtered %>% filter(rsid %in% llista_rsid)      
        }
        
        #If GWAS checkbox is selected, select those variants that have a GWAS Association Count 
        #value inside the chosen range 
        if (input$filterGWASInput == T){
            filtered <- filtered %>% filter(GWASAssocCounts >= input$GWASInput[1],
                                            GWASAssocCounts <= input$GWASInput[2])
        }
        
        #If VEP checkbox is selected, select those variants with at least one of the VEP Consequences chosen
        if (input$filterVEPInput == T){
            separatedVEP <- separate_rows(filtered, VEPConseqs, sep = ',')
            ids_filter_VEP <- separatedVEP %>% filter(VEPConseqs %in% input$VEPInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_VEP$rsid)
        }
        
        #If ClinVar checkbox is selected, select those variants with at least one of the Clinical Significances chosen
        if (input$filterClinVarInput == T){
            separatedCV <- separate_rows(filtered, ClinicalSignificances, sep = ', ')
            ids_filter_CV <- separatedCV %>% filter(ClinicalSignificances %in% input$ClinVarInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_CV$rsid)
        }
        
        #If Regulome checkbox is selected, select those variants with a regulome rank score value
        #inside the chosen range
        if (input$filterRegulomeInput == T){
            filtered <- filtered %>% filter(RegulomeImportance >= match(input$RegulomeInput[1], regulomeLabels),
                                            RegulomeImportance <= match(input$RegulomeInput[2], regulomeLabels))
        }
        
        #If DSI checbox is selected, select those variants with a DSI value inside the chosen range
        if (input$filterDSIInput == T){
            filtered <- filtered %>% filter (`max(DSI)` >= input$DSIInput[1],
                                             `max(DSI)` <= input$DSIInput[2])
        }
        
        #If DPI checbox is selected, select those variants with a DPI value inside the chosen range
        if (input$filterDPIInput == T){
            filtered <- filtered %>% filter (`max(DPI)` >= input$DPIInput[1],
                                             `max(DPI)` <= input$DPIInput[2])
        }
        
        #If score checbox is selected, select those variants with a DisGeNET score inside the chosen range
        if (input$filterScoreInput == T){
            filtered <- filtered %>% filter (`max(score)` >= input$DGNscoreInput[1],
                                             `max(score)` <= input$DGNscoreInput[2])
        }
        
        #If N.of PubMed IDs checbox is selected, select those variants with a number of 
        #PubMed IDs value inside the chosen range
        if (input$filterNofPmidsInput == T){
            filtered <- filtered %>% filter (`max(NofPmids)` >= input$NofPmidsInput[1],
                                             `max(NofPmids)` <= input$NofPmidsInput[2])
        }
        
        #If Age filter checbox is selected, select those variants with a Age mean and score value, 
        #of the chosen clock model, inside the chosen ranges
        if (input$filterAgeInput == T){
            if ('Mut' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Mut >= input$AgeMeanInput[1],
                                                AgeMean_Mut <= input$AgeMeanInput[2],
                                                QualScore_Mut >= input$qualScoreInput[1],
                                                QualScore_Mut <= input$qualScoreInput[2])
            }
            if ('Rec' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Rec >= input$AgeMeanInput[1],
                                                AgeMean_Rec <= input$AgeMeanInput[2],
                                                QualScore_Rec >= input$qualScoreInput[1],
                                                QualScore_Rec <= input$qualScoreInput[2])
            }
            if ('Jnt' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Jnt >= input$AgeMeanInput[1],
                                                AgeMean_Jnt <= input$AgeMeanInput[2],
                                                QualScore_Jnt >= input$qualScoreInput[1],
                                                QualScore_Jnt <= input$qualScoreInput[2])
            }
        }
        
        #Print a text line with how many results were found
        return(paste(nrow(filtered), 'results were found.'))
        
    })
    
    #iHS PLOT 
    output$plotIHS <- renderPlot({
        
        #Filter the original data by position and save it in filtered
        filtered <- dades %>% filter(chr %in% input$chrInput,
                                     physicalPos >= input$minPosInput,
                                     physicalPos <= input$maxPosInput)
        
        #if extreme values checkbox is selected, select those variants that have 
        #an extreme value in at least one of the populations selected
        if (input$extremeValuesInput == TRUE){
            for (POP in pops){
                if (POP %in% input$popsInput){
                    llista_rsid <- c(llista_rsid, 
                                     filtered %>% filter(!is.na(filtered[[paste0('extremevalue_',POP)]])) %>%
                                         select(rsid))
                        
                    }
            }
                filtered <- filtered %>% filter(rsid %in% llista_rsid)      
        }
        
        #If GWAS checkbox is selected, select those variants that have a GWAS Association Count 
        #value inside the chosen range 
        if (input$filterGWASInput == T){
            filtered <- filtered %>% filter(GWASAssocCounts >= input$GWASInput[1],
                                            GWASAssocCounts <= input$GWASInput[2])
        }
        
        #If VEP checkbox is selected, select those variants with at least one of the VEP Consequences chosen
        if (input$filterVEPInput == T){
            separatedVEP <- separate_rows(filtered, VEPConseqs, sep = ',')
            ids_filter_VEP <- separatedVEP %>% filter(VEPConseqs %in% input$VEPInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_VEP$rsid)
        }
        
        #If ClinVar checkbox is selected, select those variants with at least one of the Clinical Significances chosen
        if (input$filterClinVarInput == T){
            separatedCV <- separate_rows(filtered, ClinicalSignificances, sep = ', ')
            ids_filter_CV <- separatedCV %>% filter(ClinicalSignificances %in% input$ClinVarInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_CV$rsid)
        }
        
        #If Regulome checkbox is selected, select those variants with a regulome rank score value
        #inside the chosen range
        if (input$filterRegulomeInput == T){
            filtered <- filtered %>% filter(RegulomeImportance >= match(input$RegulomeInput[1], regulomeLabels),
                                            RegulomeImportance <= match(input$RegulomeInput[2], regulomeLabels))
        }
        
        #If DSI checbox is selected, select those variants with a DSI value inside the chosen range
        if (input$filterDSIInput == T){
            filtered <- filtered %>% filter (`max(DSI)` >= input$DSIInput[1],
                                             `max(DSI)` <= input$DSIInput[2])
        }
        
        #If DPI checbox is selected, select those variants with a DPI value inside the chosen range
        if (input$filterDPIInput == T){
            filtered <- filtered %>% filter (`max(DPI)` >= input$DPIInput[1],
                                             `max(DPI)` <= input$DPIInput[2])
        }
        
        #If score checbox is selected, select those variants with a DisGeNET score inside the chosen range
        if (input$filterScoreInput == T){
            filtered <- filtered %>% filter (`max(score)` >= input$DGNscoreInput[1],
                                             `max(score)` <= input$DGNscoreInput[2])
        }
        
        #If N.of PubMed IDs checbox is selected, select those variants with a number of 
        #PubMed IDs value inside the chosen range
        if (input$filterNofPmidsInput == T){
            filtered <- filtered %>% filter (`max(NofPmids)` >= input$NofPmidsInput[1],
                                             `max(NofPmids)` <= input$NofPmidsInput[2])
        }
        
        #If Age filter checbox is selected, select those variants with a Age mean and score value, 
        #of the chosen clock model, inside the chosen ranges
        if (input$filterAgeInput == T){
            if ('Mut' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Mut >= input$AgeMeanInput[1],
                                                AgeMean_Mut <= input$AgeMeanInput[2],
                                                QualScore_Mut >= input$qualScoreInput[1],
                                                QualScore_Mut <= input$qualScoreInput[2])
            }
            if ('Rec' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Rec >= input$AgeMeanInput[1],
                                                AgeMean_Rec <= input$AgeMeanInput[2],
                                                QualScore_Rec >= input$qualScoreInput[1],
                                                QualScore_Rec <= input$qualScoreInput[2])
            }
            if ('Jnt' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Jnt >= input$AgeMeanInput[1],
                                                AgeMean_Jnt <= input$AgeMeanInput[2],
                                                QualScore_Jnt >= input$qualScoreInput[1],
                                                QualScore_Jnt <= input$qualScoreInput[2])
            }
        }
        
        #Select the rsid and all iHS columns
        filtered <- filtered %>% select(rsid, starts_with('standiHS_'))
        #Transform the data from wide to long format 
        filtered_long <- melt(setDT(filtered), id.vars = "rsid", variable.name = "iHS")
        
        #Design the plot (x axis = iHS values; y axis = count)
        o <- ggplot(filtered_long, aes(x=value, y=..count.., fill = ..count..)) + geom_histogram() 
        o + scale_fill_gradient(low='skyblue', high = 'purple')
        
    })
    

    
    #GWAS ASSOCIATION COUNTS PLOT
    output$plotGWAS <- renderPlot({
        
        #Filter the original data by position and save it in filtered
        filtered <- dades %>% filter(chr %in% input$chrInput,
                                     physicalPos >= input$minPosInput,
                                     physicalPos <= input$maxPosInput)
        
        #if extreme values checkbox is selected, select those variants that have 
        #an extreme value in at least one of the populations selected
        if (input$extremeValuesInput == TRUE){
            for (POP in pops){
                if (POP %in% input$popsInput){
                    llista_rsid <- c(llista_rsid, 
                                     filtered %>% filter(!is.na(filtered[[paste0('extremevalue_',POP)]])) %>%
                                         select(rsid))
                        
                    }
            }
                filtered <- filtered %>% filter(rsid %in% llista_rsid)      
        }
        
        #If GWAS checkbox is selected, select those variants that have a GWAS Association Count 
        #value inside the chosen range 
        if (input$filterGWASInput == T){
            filtered <- filtered %>% filter(GWASAssocCounts >= input$GWASInput[1],
                                            GWASAssocCounts <= input$GWASInput[2])
        }
        
        #If VEP checkbox is selected, select those variants with at least one of the VEP Consequences chosen
        if (input$filterVEPInput == T){
            separatedVEP <- separate_rows(filtered, VEPConseqs, sep = ',')
            ids_filter_VEP <- separatedVEP %>% filter(VEPConseqs %in% input$VEPInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_VEP$rsid)
        }
        
        #If ClinVar checkbox is selected, select those variants with at least one of the Clinical Significances chosen
        if (input$filterClinVarInput == T){
            separatedCV <- separate_rows(filtered, ClinicalSignificances, sep = ', ')
            ids_filter_CV <- separatedCV %>% filter(ClinicalSignificances %in% input$ClinVarInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_CV$rsid)
        }
        
        #If Regulome checkbox is selected, select those variants with a regulome rank score value
        #inside the chosen range
        if (input$filterRegulomeInput == T){
            filtered <- filtered %>% filter(RegulomeImportance >= match(input$RegulomeInput[1], regulomeLabels),
                                            RegulomeImportance <= match(input$RegulomeInput[2], regulomeLabels))
        }
        
        #If DSI checbox is selected, select those variants with a DSI value inside the chosen range
        if (input$filterDSIInput == T){
            filtered <- filtered %>% filter (`max(DSI)` >= input$DSIInput[1],
                                             `max(DSI)` <= input$DSIInput[2])
        }
        
        #If DPI checbox is selected, select those variants with a DPI value inside the chosen range
        if (input$filterDPIInput == T){
            filtered <- filtered %>% filter (`max(DPI)` >= input$DPIInput[1],
                                             `max(DPI)` <= input$DPIInput[2])
        }
        
        #If score checbox is selected, select those variants with a DisGeNET score inside the chosen range
        if (input$filterScoreInput == T){
            filtered <- filtered %>% filter (`max(score)` >= input$DGNscoreInput[1],
                                             `max(score)` <= input$DGNscoreInput[2])
        }
        
        #If N.of PubMed IDs checbox is selected, select those variants with a number of 
        #PubMed IDs value inside the chosen range
        if (input$filterNofPmidsInput == T){
            filtered <- filtered %>% filter (`max(NofPmids)` >= input$NofPmidsInput[1],
                                             `max(NofPmids)` <= input$NofPmidsInput[2])
        }
        
        #If Age filter checbox is selected, select those variants with a Age mean and score value, 
        #of the chosen clock model, inside the chosen ranges
        if (input$filterAgeInput == T){
            if ('Mut' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Mut >= input$AgeMeanInput[1],
                                                AgeMean_Mut <= input$AgeMeanInput[2],
                                                QualScore_Mut >= input$qualScoreInput[1],
                                                QualScore_Mut <= input$qualScoreInput[2])
            }
            if ('Rec' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Rec >= input$AgeMeanInput[1],
                                                AgeMean_Rec <= input$AgeMeanInput[2],
                                                QualScore_Rec >= input$qualScoreInput[1],
                                                QualScore_Rec <= input$qualScoreInput[2])
            }
            if ('Jnt' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Jnt >= input$AgeMeanInput[1],
                                                AgeMean_Jnt <= input$AgeMeanInput[2],
                                                QualScore_Jnt >= input$qualScoreInput[1],
                                                QualScore_Jnt <= input$qualScoreInput[2])
            }
        }
        
        #Design the plot
        o <- ggplot(filtered, aes(x=GWASAssocCounts, y=..count.., fill = ..count..)) + geom_histogram() 
        o + scale_fill_gradient(low='skyblue', high = 'purple')
        
    })
   
     #VEP CONSEQUENCES PLOT
    output$plotVEP <- renderPlot({
        
        #Filter the original data by position and save it in filtered
        filtered <- dades %>% filter(chr %in% input$chrInput,
                                     physicalPos >= input$minPosInput,
                                     physicalPos <= input$maxPosInput)
        
        #if extreme values checkbox is selected, select those variants that have 
        #an extreme value in at least one of the populations selected
        if (input$extremeValuesInput == TRUE){
            for (POP in pops){
                if (POP %in% input$popsInput){
                    llista_rsid <- c(llista_rsid, 
                                     filtered %>% filter(!is.na(filtered[[paste0('extremevalue_',POP)]])) %>%
                                         select(rsid))
                        
                    }
            }
                filtered <- filtered %>% filter(rsid %in% llista_rsid)      
        }
        
        #If GWAS checkbox is selected, select those variants that have a GWAS Association Count 
        #value inside the chosen range 
        if (input$filterGWASInput == T){
            filtered <- filtered %>% filter(GWASAssocCounts >= input$GWASInput[1],
                                            GWASAssocCounts <= input$GWASInput[2])
        }
        
        #If VEP checkbox is selected, select those variants with at least one of the VEP Consequences chosen
        if (input$filterVEPInput == T){
            separatedVEP <- separate_rows(filtered, VEPConseqs, sep = ',')
            ids_filter_VEP <- separatedVEP %>% filter(VEPConseqs %in% input$VEPInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_VEP$rsid)
        }
        
        #If ClinVar checkbox is selected, select those variants with at least one of the Clinical Significances chosen
        if (input$filterClinVarInput == T){
            separatedCV <- separate_rows(filtered, ClinicalSignificances, sep = ', ')
            ids_filter_CV <- separatedCV %>% filter(ClinicalSignificances %in% input$ClinVarInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_CV$rsid)
        }
        
        #If Regulome checkbox is selected, select those variants with a regulome rank score value
        #inside the chosen range
        if (input$filterRegulomeInput == T){
            filtered <- filtered %>% filter(RegulomeImportance >= match(input$RegulomeInput[1], regulomeLabels),
                                            RegulomeImportance <= match(input$RegulomeInput[2], regulomeLabels))
        }
        
        #If DSI checbox is selected, select those variants with a DSI value inside the chosen range
        if (input$filterDSIInput == T){
            filtered <- filtered %>% filter (`max(DSI)` >= input$DSIInput[1],
                                             `max(DSI)` <= input$DSIInput[2])
        }
        
        #If DPI checbox is selected, select those variants with a DPI value inside the chosen range
        if (input$filterDPIInput == T){
            filtered <- filtered %>% filter (`max(DPI)` >= input$DPIInput[1],
                                             `max(DPI)` <= input$DPIInput[2])
        }
        
        #If score checbox is selected, select those variants with a DisGeNET score inside the chosen range
        if (input$filterScoreInput == T){
            filtered <- filtered %>% filter (`max(score)` >= input$DGNscoreInput[1],
                                             `max(score)` <= input$DGNscoreInput[2])
        }
        
        #If N.of PubMed IDs checbox is selected, select those variants with a number of 
        #PubMed IDs value inside the chosen range
        if (input$filterNofPmidsInput == T){
            filtered <- filtered %>% filter (`max(NofPmids)` >= input$NofPmidsInput[1],
                                             `max(NofPmids)` <= input$NofPmidsInput[2])
        }
        
        #If Age filter checbox is selected, select those variants with a Age mean and score value, 
        #of the chosen clock model, inside the chosen ranges
        if (input$filterAgeInput == T){
            if ('Mut' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Mut >= input$AgeMeanInput[1],
                                                AgeMean_Mut <= input$AgeMeanInput[2],
                                                QualScore_Mut >= input$qualScoreInput[1],
                                                QualScore_Mut <= input$qualScoreInput[2])
            }
            if ('Rec' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Rec >= input$AgeMeanInput[1],
                                                AgeMean_Rec <= input$AgeMeanInput[2],
                                                QualScore_Rec >= input$qualScoreInput[1],
                                                QualScore_Rec <= input$qualScoreInput[2])
            }
            if ('Jnt' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Jnt >= input$AgeMeanInput[1],
                                                AgeMean_Jnt <= input$AgeMeanInput[2],
                                                QualScore_Jnt >= input$qualScoreInput[1],
                                                QualScore_Jnt <= input$qualScoreInput[2])
            }
        }
        
        #separate the rows that have more than one VEP Consequence to count them independently 
        filter_separat <- separate_rows(filtered, VEPConseqs, sep = ',')
        
        #Design the plot 
        o <- ggplot(filter_separat, aes(x=VEPConseqs, stat="count", fill = ..count..)) + geom_bar() 
        o + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) + scale_fill_gradient(low = 'skyblue', high = 'purple')
        
    })
    
    #CLINVAR PLOT
    output$plotClinVar <- renderPlot({
        
        #Filter the original data by position and save it in filtered
        filtered <- dades %>% filter(chr %in% input$chrInput,
                                     physicalPos >= input$minPosInput,
                                     physicalPos <= input$maxPosInput)
        
        #if extreme values checkbox is selected, select those variants that have 
        #an extreme value in at least one of the populations selected
        if (input$extremeValuesInput == TRUE){
            for (POP in pops){
                if (POP %in% input$popsInput){
                    llista_rsid <- c(llista_rsid, 
                                     filtered %>% filter(!is.na(filtered[[paste0('extremevalue_',POP)]])) %>%
                                         select(rsid))
                        
                    }
            }
                filtered <- filtered %>% filter(rsid %in% llista_rsid)      
        }
        
        #If GWAS checkbox is selected, select those variants that have a GWAS Association Count 
        #value inside the chosen range 
        if (input$filterGWASInput == T){
            filtered <- filtered %>% filter(GWASAssocCounts >= input$GWASInput[1],
                                            GWASAssocCounts <= input$GWASInput[2])
        }
        
        #If VEP checkbox is selected, select those variants with at least one of the VEP Consequences chosen
        if (input$filterVEPInput == T){
            separatedVEP <- separate_rows(filtered, VEPConseqs, sep = ',')
            ids_filter_VEP <- separatedVEP %>% filter(VEPConseqs %in% input$VEPInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_VEP$rsid)
        }
        
        #If ClinVar checkbox is selected, select those variants with at least one of the Clinical Significances chosen
        if (input$filterClinVarInput == T){
            separatedCV <- separate_rows(filtered, ClinicalSignificances, sep = ', ')
            ids_filter_CV <- separatedCV %>% filter(ClinicalSignificances %in% input$ClinVarInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_CV$rsid)
        }
        
        #If Regulome checkbox is selected, select those variants with a regulome rank score value
        #inside the chosen range
        if (input$filterRegulomeInput == T){
            filtered <- filtered %>% filter(RegulomeImportance >= match(input$RegulomeInput[1], regulomeLabels),
                                            RegulomeImportance <= match(input$RegulomeInput[2], regulomeLabels))
        }
        
        #If DSI checbox is selected, select those variants with a DSI value inside the chosen range
        if (input$filterDSIInput == T){
            filtered <- filtered %>% filter (`max(DSI)` >= input$DSIInput[1],
                                             `max(DSI)` <= input$DSIInput[2])
        }
        
        #If DPI checbox is selected, select those variants with a DPI value inside the chosen range
        if (input$filterDPIInput == T){
            filtered <- filtered %>% filter (`max(DPI)` >= input$DPIInput[1],
                                             `max(DPI)` <= input$DPIInput[2])
        }
        
        #If score checbox is selected, select those variants with a DisGeNET score inside the chosen range
        if (input$filterScoreInput == T){
            filtered <- filtered %>% filter (`max(score)` >= input$DGNscoreInput[1],
                                             `max(score)` <= input$DGNscoreInput[2])
        }
        
        #If N.of PubMed IDs checbox is selected, select those variants with a number of 
        #PubMed IDs value inside the chosen range
        if (input$filterNofPmidsInput == T){
            filtered <- filtered %>% filter (`max(NofPmids)` >= input$NofPmidsInput[1],
                                             `max(NofPmids)` <= input$NofPmidsInput[2])
        }
        
        #If Age filter checbox is selected, select those variants with a Age mean and score value, 
        #of the chosen clock model, inside the chosen ranges
        if (input$filterAgeInput == T){
            if ('Mut' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Mut >= input$AgeMeanInput[1],
                                                AgeMean_Mut <= input$AgeMeanInput[2],
                                                QualScore_Mut >= input$qualScoreInput[1],
                                                QualScore_Mut <= input$qualScoreInput[2])
            }
            if ('Rec' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Rec >= input$AgeMeanInput[1],
                                                AgeMean_Rec <= input$AgeMeanInput[2],
                                                QualScore_Rec >= input$qualScoreInput[1],
                                                QualScore_Rec <= input$qualScoreInput[2])
            }
            if ('Jnt' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Jnt >= input$AgeMeanInput[1],
                                                AgeMean_Jnt <= input$AgeMeanInput[2],
                                                QualScore_Jnt >= input$qualScoreInput[1],
                                                QualScore_Jnt <= input$qualScoreInput[2])
            }
        }
        
        #Separate the rows that have more than one clinical significance to count them independently
        filter_separat <- separate_rows(filtered, ClinicalSignificances, sep = ', ')
       
        #Design the plot
        o <- ggplot(filter_separat, aes(x=ClinicalSignificances, stat="count", fill = ..count..)) + geom_bar() 
        o + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) + scale_fill_gradient(low = 'skyblue', high = 'purple')
        
    })
    
    #REGULOME PLOT
    output$plotRegulome <- renderPlot({
        
        #Filter the original data by position and save it in filtered
        filtered <- dades %>% filter(chr %in% input$chrInput,
                                     physicalPos >= input$minPosInput,
                                     physicalPos <= input$maxPosInput)
        
        #if extreme values checkbox is selected, select those variants that have 
        #an extreme value in at least one of the populations selected
        if (input$extremeValuesInput == TRUE){
            for (POP in pops){
                if (POP %in% input$popsInput){
                    llista_rsid <- c(llista_rsid, 
                                     filtered %>% filter(!is.na(filtered[[paste0('extremevalue_',POP)]])) %>%
                                         select(rsid))
                        
                    }
            }
                filtered <- filtered %>% filter(rsid %in% llista_rsid)      
        }
        
        #If GWAS checkbox is selected, select those variants that have a GWAS Association Count 
        #value inside the chosen range 
        if (input$filterGWASInput == T){
            filtered <- filtered %>% filter(GWASAssocCounts >= input$GWASInput[1],
                                            GWASAssocCounts <= input$GWASInput[2])
        }
        
        #If VEP checkbox is selected, select those variants with at least one of the VEP Consequences chosen
        if (input$filterVEPInput == T){
            separatedVEP <- separate_rows(filtered, VEPConseqs, sep = ',')
            ids_filter_VEP <- separatedVEP %>% filter(VEPConseqs %in% input$VEPInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_VEP$rsid)
        }
        
        #If ClinVar checkbox is selected, select those variants with at least one of the Clinical Significances chosen
        if (input$filterClinVarInput == T){
            separatedCV <- separate_rows(filtered, ClinicalSignificances, sep = ', ')
            ids_filter_CV <- separatedCV %>% filter(ClinicalSignificances %in% input$ClinVarInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_CV$rsid)
        }
        
        #If Regulome checkbox is selected, select those variants with a regulome rank score value
        #inside the chosen range
        if (input$filterRegulomeInput == T){
            filtered <- filtered %>% filter(RegulomeImportance >= match(input$RegulomeInput[1], regulomeLabels),
                                            RegulomeImportance <= match(input$RegulomeInput[2], regulomeLabels))
        }
        
        #If DSI checbox is selected, select those variants with a DSI value inside the chosen range
        if (input$filterDSIInput == T){
            filtered <- filtered %>% filter (`max(DSI)` >= input$DSIInput[1],
                                             `max(DSI)` <= input$DSIInput[2])
        }
        
        #If DPI checbox is selected, select those variants with a DPI value inside the chosen range
        if (input$filterDPIInput == T){
            filtered <- filtered %>% filter (`max(DPI)` >= input$DPIInput[1],
                                             `max(DPI)` <= input$DPIInput[2])
        }
        
        #If score checbox is selected, select those variants with a DisGeNET score inside the chosen range
        if (input$filterScoreInput == T){
            filtered <- filtered %>% filter (`max(score)` >= input$DGNscoreInput[1],
                                             `max(score)` <= input$DGNscoreInput[2])
        }
        
        #If N.of PubMed IDs checbox is selected, select those variants with a number of 
        #PubMed IDs value inside the chosen range
        if (input$filterNofPmidsInput == T){
            filtered <- filtered %>% filter (`max(NofPmids)` >= input$NofPmidsInput[1],
                                             `max(NofPmids)` <= input$NofPmidsInput[2])
        }
        
        #If Age filter checbox is selected, select those variants with a Age mean and score value, 
        #of the chosen clock model, inside the chosen ranges
        if (input$filterAgeInput == T){
            if ('Mut' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Mut >= input$AgeMeanInput[1],
                                                AgeMean_Mut <= input$AgeMeanInput[2],
                                                QualScore_Mut >= input$qualScoreInput[1],
                                                QualScore_Mut <= input$qualScoreInput[2])
            }
            if ('Rec' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Rec >= input$AgeMeanInput[1],
                                                AgeMean_Rec <= input$AgeMeanInput[2],
                                                QualScore_Rec >= input$qualScoreInput[1],
                                                QualScore_Rec <= input$qualScoreInput[2])
            }
            if ('Jnt' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Jnt >= input$AgeMeanInput[1],
                                                AgeMean_Jnt <= input$AgeMeanInput[2],
                                                QualScore_Jnt >= input$qualScoreInput[1],
                                                QualScore_Jnt <= input$qualScoreInput[2])
            }
        }
        
        #Design the plot
        o <- ggplot(filtered, aes(x=RegulomeScore, stat="count", fill = ..count..)) + geom_bar() 
        o + scale_fill_gradient(low = 'skyblue', high = 'purple')
        
    })
    
    
    #DSI PLOT
    output$plotDSI <- renderPlot({
       
        #Filter the original data by position and save it in filtered
        filtered <- dades %>% filter(chr %in% input$chrInput,
                                     physicalPos >= input$minPosInput,
                                     physicalPos <= input$maxPosInput)
        
        #if extreme values checkbox is selected, select those variants that have 
        #an extreme value in at least one of the populations selected
        if (input$extremeValuesInput == TRUE){
            for (POP in pops){
                if (POP %in% input$popsInput){
                    llista_rsid <- c(llista_rsid, 
                                     filtered %>% filter(!is.na(filtered[[paste0('extremevalue_',POP)]])) %>%
                                         select(rsid))
                        
                    }
            }
                filtered <- filtered %>% filter(rsid %in% llista_rsid)      
        }
        
        #If GWAS checkbox is selected, select those variants that have a GWAS Association Count 
        #value inside the chosen range 
        if (input$filterGWASInput == T){
            filtered <- filtered %>% filter(GWASAssocCounts >= input$GWASInput[1],
                                            GWASAssocCounts <= input$GWASInput[2])
        }
        
        #If VEP checkbox is selected, select those variants with at least one of the VEP Consequences chosen
        if (input$filterVEPInput == T){
            separatedVEP <- separate_rows(filtered, VEPConseqs, sep = ',')
            ids_filter_VEP <- separatedVEP %>% filter(VEPConseqs %in% input$VEPInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_VEP$rsid)
        }
        
        #If ClinVar checkbox is selected, select those variants with at least one of the Clinical Significances chosen
        if (input$filterClinVarInput == T){
            separatedCV <- separate_rows(filtered, ClinicalSignificances, sep = ', ')
            ids_filter_CV <- separatedCV %>% filter(ClinicalSignificances %in% input$ClinVarInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_CV$rsid)
        }
        
        #If Regulome checkbox is selected, select those variants with a regulome rank score value
        #inside the chosen range
        if (input$filterRegulomeInput == T){
            filtered <- filtered %>% filter(RegulomeImportance >= match(input$RegulomeInput[1], regulomeLabels),
                                            RegulomeImportance <= match(input$RegulomeInput[2], regulomeLabels))
        }
        
        #If DSI checbox is selected, select those variants with a DSI value inside the chosen range
        if (input$filterDSIInput == T){
            filtered <- filtered %>% filter (`max(DSI)` >= input$DSIInput[1],
                                             `max(DSI)` <= input$DSIInput[2])
        }
        
        #If DPI checbox is selected, select those variants with a DPI value inside the chosen range
        if (input$filterDPIInput == T){
            filtered <- filtered %>% filter (`max(DPI)` >= input$DPIInput[1],
                                             `max(DPI)` <= input$DPIInput[2])
        }
        
        #If score checbox is selected, select those variants with a DisGeNET score inside the chosen range
        if (input$filterScoreInput == T){
            filtered <- filtered %>% filter (`max(score)` >= input$DGNscoreInput[1],
                                             `max(score)` <= input$DGNscoreInput[2])
        }
        
        #If N.of PubMed IDs checbox is selected, select those variants with a number of 
        #PubMed IDs value inside the chosen range
        if (input$filterNofPmidsInput == T){
            filtered <- filtered %>% filter (`max(NofPmids)` >= input$NofPmidsInput[1],
                                             `max(NofPmids)` <= input$NofPmidsInput[2])
        }
        
        #If Age filter checbox is selected, select those variants with a Age mean and score value, 
        #of the chosen clock model, inside the chosen ranges
        if (input$filterAgeInput == T){
            if ('Mut' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Mut >= input$AgeMeanInput[1],
                                                AgeMean_Mut <= input$AgeMeanInput[2],
                                                QualScore_Mut >= input$qualScoreInput[1],
                                                QualScore_Mut <= input$qualScoreInput[2])
            }
            if ('Rec' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Rec >= input$AgeMeanInput[1],
                                                AgeMean_Rec <= input$AgeMeanInput[2],
                                                QualScore_Rec >= input$qualScoreInput[1],
                                                QualScore_Rec <= input$qualScoreInput[2])
            }
            if ('Jnt' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Jnt >= input$AgeMeanInput[1],
                                                AgeMean_Jnt <= input$AgeMeanInput[2],
                                                QualScore_Jnt >= input$qualScoreInput[1],
                                                QualScore_Jnt <= input$qualScoreInput[2])
            }
        }
        
        #Design the plot
        o <- ggplot(filtered, aes(x=`max(DSI)`, y=..count.., fill = ..count..)) + geom_histogram() 
        o + scale_fill_gradient(low='skyblue', high = 'purple')
        
    })
    #DPI PLOT
    output$plotDPI <- renderPlot({
        
        #Filter the original data by position and save it in filtered
        filtered <- dades %>% filter(chr %in% input$chrInput,
                                     physicalPos >= input$minPosInput,
                                     physicalPos <= input$maxPosInput)
        
        #if extreme values checkbox is selected, select those variants that have 
        #an extreme value in at least one of the populations selected
        if (input$extremeValuesInput == TRUE){
            for (POP in pops){
                if (POP %in% input$popsInput){
                    llista_rsid <- c(llista_rsid, 
                                     filtered %>% filter(!is.na(filtered[[paste0('extremevalue_',POP)]])) %>%
                                         select(rsid))
                        
                    }
            }
                filtered <- filtered %>% filter(rsid %in% llista_rsid)      
        }
        
        #If GWAS checkbox is selected, select those variants that have a GWAS Association Count 
        #value inside the chosen range 
        if (input$filterGWASInput == T){
            filtered <- filtered %>% filter(GWASAssocCounts >= input$GWASInput[1],
                                            GWASAssocCounts <= input$GWASInput[2])
        }
        
        #If VEP checkbox is selected, select those variants with at least one of the VEP Consequences chosen
        if (input$filterVEPInput == T){
            separatedVEP <- separate_rows(filtered, VEPConseqs, sep = ',')
            ids_filter_VEP <- separatedVEP %>% filter(VEPConseqs %in% input$VEPInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_VEP$rsid)
        }
        
        #If ClinVar checkbox is selected, select those variants with at least one of the Clinical Significances chosen
        if (input$filterClinVarInput == T){
            separatedCV <- separate_rows(filtered, ClinicalSignificances, sep = ', ')
            ids_filter_CV <- separatedCV %>% filter(ClinicalSignificances %in% input$ClinVarInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_CV$rsid)
        }
        
        #If Regulome checkbox is selected, select those variants with a regulome rank score value
        #inside the chosen range
        if (input$filterRegulomeInput == T){
            filtered <- filtered %>% filter(RegulomeImportance >= match(input$RegulomeInput[1], regulomeLabels),
                                            RegulomeImportance <= match(input$RegulomeInput[2], regulomeLabels))
        }
        
        #If DSI checbox is selected, select those variants with a DSI value inside the chosen range
        if (input$filterDSIInput == T){
            filtered <- filtered %>% filter (`max(DSI)` >= input$DSIInput[1],
                                             `max(DSI)` <= input$DSIInput[2])
        }
        
        #If DPI checbox is selected, select those variants with a DPI value inside the chosen range
        if (input$filterDPIInput == T){
            filtered <- filtered %>% filter (`max(DPI)` >= input$DPIInput[1],
                                             `max(DPI)` <= input$DPIInput[2])
        }
        
        #If score checbox is selected, select those variants with a DisGeNET score inside the chosen range
        if (input$filterScoreInput == T){
            filtered <- filtered %>% filter (`max(score)` >= input$DGNscoreInput[1],
                                             `max(score)` <= input$DGNscoreInput[2])
        }
        
        #If N.of PubMed IDs checbox is selected, select those variants with a number of 
        #PubMed IDs value inside the chosen range
        if (input$filterNofPmidsInput == T){
            filtered <- filtered %>% filter (`max(NofPmids)` >= input$NofPmidsInput[1],
                                             `max(NofPmids)` <= input$NofPmidsInput[2])
        }
        
        #If Age filter checbox is selected, select those variants with a Age mean and score value, 
        #of the chosen clock model, inside the chosen ranges
        if (input$filterAgeInput == T){
            if ('Mut' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Mut >= input$AgeMeanInput[1],
                                                AgeMean_Mut <= input$AgeMeanInput[2],
                                                QualScore_Mut >= input$qualScoreInput[1],
                                                QualScore_Mut <= input$qualScoreInput[2])
            }
            if ('Rec' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Rec >= input$AgeMeanInput[1],
                                                AgeMean_Rec <= input$AgeMeanInput[2],
                                                QualScore_Rec >= input$qualScoreInput[1],
                                                QualScore_Rec <= input$qualScoreInput[2])
            }
            if ('Jnt' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Jnt >= input$AgeMeanInput[1],
                                                AgeMean_Jnt <= input$AgeMeanInput[2],
                                                QualScore_Jnt >= input$qualScoreInput[1],
                                                QualScore_Jnt <= input$qualScoreInput[2])
            }
        }
        
        #Design the plot
        o <- ggplot(filtered, aes(x=`max(DPI)`, y=..count.., fill = ..count..)) + geom_histogram() 
        o + scale_fill_gradient(low='skyblue', high = 'purple')
        
    })

    #DISGENET SCORE PLOT
    output$plotScore <- renderPlot({
        
        #Filter the original data by position and save it in filtered
        filtered <- dades %>% filter(chr %in% input$chrInput,
                                     physicalPos >= input$minPosInput,
                                     physicalPos <= input$maxPosInput)
        
        #if extreme values checkbox is selected, select those variants that have 
        #an extreme value in at least one of the populations selected
        if (input$extremeValuesInput == TRUE){
            for (POP in pops){
                if (POP %in% input$popsInput){
                    llista_rsid <- c(llista_rsid, 
                                     filtered %>% filter(!is.na(filtered[[paste0('extremevalue_',POP)]])) %>%
                                         select(rsid))
                        
                    }
            }
                filtered <- filtered %>% filter(rsid %in% llista_rsid)      
        }
        
        #If GWAS checkbox is selected, select those variants that have a GWAS Association Count 
        #value inside the chosen range 
        if (input$filterGWASInput == T){
            filtered <- filtered %>% filter(GWASAssocCounts >= input$GWASInput[1],
                                            GWASAssocCounts <= input$GWASInput[2])
        }
        
        #If VEP checkbox is selected, select those variants with at least one of the VEP Consequences chosen
        if (input$filterVEPInput == T){
            separatedVEP <- separate_rows(filtered, VEPConseqs, sep = ',')
            ids_filter_VEP <- separatedVEP %>% filter(VEPConseqs %in% input$VEPInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_VEP$rsid)
        }
        
        #If ClinVar checkbox is selected, select those variants with at least one of the Clinical Significances chosen
        if (input$filterClinVarInput == T){
            separatedCV <- separate_rows(filtered, ClinicalSignificances, sep = ', ')
            ids_filter_CV <- separatedCV %>% filter(ClinicalSignificances %in% input$ClinVarInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_CV$rsid)
        }
        
        #If Regulome checkbox is selected, select those variants with a regulome rank score value
        #inside the chosen range
        if (input$filterRegulomeInput == T){
            filtered <- filtered %>% filter(RegulomeImportance >= match(input$RegulomeInput[1], regulomeLabels),
                                            RegulomeImportance <= match(input$RegulomeInput[2], regulomeLabels))
        }
        
        #If DSI checbox is selected, select those variants with a DSI value inside the chosen range
        if (input$filterDSIInput == T){
            filtered <- filtered %>% filter (`max(DSI)` >= input$DSIInput[1],
                                             `max(DSI)` <= input$DSIInput[2])
        }
        
        #If DPI checbox is selected, select those variants with a DPI value inside the chosen range
        if (input$filterDPIInput == T){
            filtered <- filtered %>% filter (`max(DPI)` >= input$DPIInput[1],
                                             `max(DPI)` <= input$DPIInput[2])
        }
        
        #If score checbox is selected, select those variants with a DisGeNET score inside the chosen range
        if (input$filterScoreInput == T){
            filtered <- filtered %>% filter (`max(score)` >= input$DGNscoreInput[1],
                                             `max(score)` <= input$DGNscoreInput[2])
        }
        
        #If N.of PubMed IDs checbox is selected, select those variants with a number of 
        #PubMed IDs value inside the chosen range
        if (input$filterNofPmidsInput == T){
            filtered <- filtered %>% filter (`max(NofPmids)` >= input$NofPmidsInput[1],
                                             `max(NofPmids)` <= input$NofPmidsInput[2])
        }
        
        #If Age filter checbox is selected, select those variants with a Age mean and score value, 
        #of the chosen clock model, inside the chosen ranges
        if (input$filterAgeInput == T){
            if ('Mut' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Mut >= input$AgeMeanInput[1],
                                                AgeMean_Mut <= input$AgeMeanInput[2],
                                                QualScore_Mut >= input$qualScoreInput[1],
                                                QualScore_Mut <= input$qualScoreInput[2])
            }
            if ('Rec' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Rec >= input$AgeMeanInput[1],
                                                AgeMean_Rec <= input$AgeMeanInput[2],
                                                QualScore_Rec >= input$qualScoreInput[1],
                                                QualScore_Rec <= input$qualScoreInput[2])
            }
            if ('Jnt' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Jnt >= input$AgeMeanInput[1],
                                                AgeMean_Jnt <= input$AgeMeanInput[2],
                                                QualScore_Jnt >= input$qualScoreInput[1],
                                                QualScore_Jnt <= input$qualScoreInput[2])
            }
        }
        
        #Design the plot
        o <- ggplot(filtered, aes(x=`max(score)`, y=..count.., fill = ..count..)) + geom_histogram() 
        o + scale_fill_gradient(low='skyblue', high = 'purple')
        
    })
    
    #NUMBER OF PUBMED IDS PLOT
    output$plotNofPmids <- renderPlot({
        
        #Filter the original data by position and save it in filtered
        filtered <- dades %>% filter(chr %in% input$chrInput,
                                     physicalPos >= input$minPosInput,
                                     physicalPos <= input$maxPosInput)
        
        #if extreme values checkbox is selected, select those variants that have 
        #an extreme value in at least one of the populations selected
        if (input$extremeValuesInput == TRUE){
            for (POP in pops){
                if (POP %in% input$popsInput){
                    llista_rsid <- c(llista_rsid, 
                                     filtered %>% filter(!is.na(filtered[[paste0('extremevalue_',POP)]])) %>%
                                         select(rsid))
                        
                    }
            }
                filtered <- filtered %>% filter(rsid %in% llista_rsid)      
        }
        
        #If GWAS checkbox is selected, select those variants that have a GWAS Association Count 
        #value inside the chosen range 
        if (input$filterGWASInput == T){
            filtered <- filtered %>% filter(GWASAssocCounts >= input$GWASInput[1],
                                            GWASAssocCounts <= input$GWASInput[2])
        }
        
        #If VEP checkbox is selected, select those variants with at least one of the VEP Consequences chosen
        if (input$filterVEPInput == T){
            separatedVEP <- separate_rows(filtered, VEPConseqs, sep = ',')
            ids_filter_VEP <- separatedVEP %>% filter(VEPConseqs %in% input$VEPInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_VEP$rsid)
        }
        
        #If ClinVar checkbox is selected, select those variants with at least one of the Clinical Significances chosen
        if (input$filterClinVarInput == T){
            separatedCV <- separate_rows(filtered, ClinicalSignificances, sep = ', ')
            ids_filter_CV <- separatedCV %>% filter(ClinicalSignificances %in% input$ClinVarInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_CV$rsid)
        }
        
        #If Regulome checkbox is selected, select those variants with a regulome rank score value
        #inside the chosen range
        if (input$filterRegulomeInput == T){
            filtered <- filtered %>% filter(RegulomeImportance >= match(input$RegulomeInput[1], regulomeLabels),
                                            RegulomeImportance <= match(input$RegulomeInput[2], regulomeLabels))
        }
        
        #If DSI checbox is selected, select those variants with a DSI value inside the chosen range
        if (input$filterDSIInput == T){
            filtered <- filtered %>% filter (`max(DSI)` >= input$DSIInput[1],
                                             `max(DSI)` <= input$DSIInput[2])
        }
        
        #If DPI checbox is selected, select those variants with a DPI value inside the chosen range
        if (input$filterDPIInput == T){
            filtered <- filtered %>% filter (`max(DPI)` >= input$DPIInput[1],
                                             `max(DPI)` <= input$DPIInput[2])
        }
        
        #If score checbox is selected, select those variants with a DisGeNET score inside the chosen range
        if (input$filterScoreInput == T){
            filtered <- filtered %>% filter (`max(score)` >= input$DGNscoreInput[1],
                                             `max(score)` <= input$DGNscoreInput[2])
        }
        
        #If N.of PubMed IDs checbox is selected, select those variants with a number of 
        #PubMed IDs value inside the chosen range
        if (input$filterNofPmidsInput == T){
            filtered <- filtered %>% filter (`max(NofPmids)` >= input$NofPmidsInput[1],
                                             `max(NofPmids)` <= input$NofPmidsInput[2])
        }
        
        #If Age filter checbox is selected, select those variants with a Age mean and score value, 
        #of the chosen clock model, inside the chosen ranges
        if (input$filterAgeInput == T){
            if ('Mut' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Mut >= input$AgeMeanInput[1],
                                                AgeMean_Mut <= input$AgeMeanInput[2],
                                                QualScore_Mut >= input$qualScoreInput[1],
                                                QualScore_Mut <= input$qualScoreInput[2])
            }
            if ('Rec' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Rec >= input$AgeMeanInput[1],
                                                AgeMean_Rec <= input$AgeMeanInput[2],
                                                QualScore_Rec >= input$qualScoreInput[1],
                                                QualScore_Rec <= input$qualScoreInput[2])
            }
            if ('Jnt' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Jnt >= input$AgeMeanInput[1],
                                                AgeMean_Jnt <= input$AgeMeanInput[2],
                                                QualScore_Jnt >= input$qualScoreInput[1],
                                                QualScore_Jnt <= input$qualScoreInput[2])
            }
        }
        
        #Design the plot
        o <- ggplot(filtered, aes(x=`max(NofPmids)`, y=..count.., fill = ..count..)) + geom_histogram() 
        o + scale_fill_gradient(low='skyblue', high = 'purple')
        
    })
    
    #AGE MEAN (MUTATION CLOCK) PLOT
    output$plotAgeMeanMut <- renderPlot({
        
        #Filter the original data by position and save it in filtered
        filtered <- dades %>% filter(chr %in% input$chrInput,
                                     physicalPos >= input$minPosInput,
                                     physicalPos <= input$maxPosInput)
        
        #if extreme values checkbox is selected, select those variants that have 
        #an extreme value in at least one of the populations selected
        if (input$extremeValuesInput == TRUE){
            for (POP in pops){
                if (POP %in% input$popsInput){
                    llista_rsid <- c(llista_rsid, 
                                     filtered %>% filter(!is.na(filtered[[paste0('extremevalue_',POP)]])) %>%
                                         select(rsid))
                        
                    }
            }
                filtered <- filtered %>% filter(rsid %in% llista_rsid)      
        }
        
        #If GWAS checkbox is selected, select those variants that have a GWAS Association Count 
        #value inside the chosen range 
        if (input$filterGWASInput == T){
            filtered <- filtered %>% filter(GWASAssocCounts >= input$GWASInput[1],
                                            GWASAssocCounts <= input$GWASInput[2])
        }
        
        #If VEP checkbox is selected, select those variants with at least one of the VEP Consequences chosen
        if (input$filterVEPInput == T){
            separatedVEP <- separate_rows(filtered, VEPConseqs, sep = ',')
            ids_filter_VEP <- separatedVEP %>% filter(VEPConseqs %in% input$VEPInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_VEP$rsid)
        }
        
        #If ClinVar checkbox is selected, select those variants with at least one of the Clinical Significances chosen
        if (input$filterClinVarInput == T){
            separatedCV <- separate_rows(filtered, ClinicalSignificances, sep = ', ')
            ids_filter_CV <- separatedCV %>% filter(ClinicalSignificances %in% input$ClinVarInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_CV$rsid)
        }
        
        #If Regulome checkbox is selected, select those variants with a regulome rank score value
        #inside the chosen range
        if (input$filterRegulomeInput == T){
            filtered <- filtered %>% filter(RegulomeImportance >= match(input$RegulomeInput[1], regulomeLabels),
                                            RegulomeImportance <= match(input$RegulomeInput[2], regulomeLabels))
        }
        
        #If DSI checbox is selected, select those variants with a DSI value inside the chosen range
        if (input$filterDSIInput == T){
            filtered <- filtered %>% filter (`max(DSI)` >= input$DSIInput[1],
                                             `max(DSI)` <= input$DSIInput[2])
        }
        
        #If DPI checbox is selected, select those variants with a DPI value inside the chosen range
        if (input$filterDPIInput == T){
            filtered <- filtered %>% filter (`max(DPI)` >= input$DPIInput[1],
                                             `max(DPI)` <= input$DPIInput[2])
        }
        
        #If score checbox is selected, select those variants with a DisGeNET score inside the chosen range
        if (input$filterScoreInput == T){
            filtered <- filtered %>% filter (`max(score)` >= input$DGNscoreInput[1],
                                             `max(score)` <= input$DGNscoreInput[2])
        }
        
        #If N.of PubMed IDs checbox is selected, select those variants with a number of 
        #PubMed IDs value inside the chosen range
        if (input$filterNofPmidsInput == T){
            filtered <- filtered %>% filter (`max(NofPmids)` >= input$NofPmidsInput[1],
                                             `max(NofPmids)` <= input$NofPmidsInput[2])
        }
        
        #If Age filter checbox is selected, select those variants with a Age mean and score value, 
        #of the chosen clock model, inside the chosen ranges
        if (input$filterAgeInput == T){
            if ('Mut' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Mut >= input$AgeMeanInput[1],
                                                AgeMean_Mut <= input$AgeMeanInput[2],
                                                QualScore_Mut >= input$qualScoreInput[1],
                                                QualScore_Mut <= input$qualScoreInput[2])
            }
            if ('Rec' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Rec >= input$AgeMeanInput[1],
                                                AgeMean_Rec <= input$AgeMeanInput[2],
                                                QualScore_Rec >= input$qualScoreInput[1],
                                                QualScore_Rec <= input$qualScoreInput[2])
            }
            if ('Jnt' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Jnt >= input$AgeMeanInput[1],
                                                AgeMean_Jnt <= input$AgeMeanInput[2],
                                                QualScore_Jnt >= input$qualScoreInput[1],
                                                QualScore_Jnt <= input$qualScoreInput[2])
            }
        }
        
        #Design the plot
        o <- ggplot(filtered, aes(x=AgeMean_Mut, y=..count.., fill = ..count..)) + geom_histogram() 
        o + scale_fill_gradient(low='skyblue', high = 'purple')
        
    })
    
    #AGE MEAN (RECOMBINATION CLOCK) PLOT
    output$plotAgeMeanRec <- renderPlot({
        
        #Filter the original data by position and save it in filtered
        filtered <- dades %>% filter(chr %in% input$chrInput,
                                     physicalPos >= input$minPosInput,
                                     physicalPos <= input$maxPosInput)
        
        #if extreme values checkbox is selected, select those variants that have 
        #an extreme value in at least one of the populations selected
        if (input$extremeValuesInput == TRUE){
            for (POP in pops){
                if (POP %in% input$popsInput){
                    llista_rsid <- c(llista_rsid, 
                                     filtered %>% filter(!is.na(filtered[[paste0('extremevalue_',POP)]])) %>%
                                         select(rsid))
                        
                    }
            }
                filtered <- filtered %>% filter(rsid %in% llista_rsid)      
        }
        
        #If GWAS checkbox is selected, select those variants that have a GWAS Association Count 
        #value inside the chosen range 
        if (input$filterGWASInput == T){
            filtered <- filtered %>% filter(GWASAssocCounts >= input$GWASInput[1],
                                            GWASAssocCounts <= input$GWASInput[2])
        }
        
        #If VEP checkbox is selected, select those variants with at least one of the VEP Consequences chosen
        if (input$filterVEPInput == T){
            separatedVEP <- separate_rows(filtered, VEPConseqs, sep = ',')
            ids_filter_VEP <- separatedVEP %>% filter(VEPConseqs %in% input$VEPInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_VEP$rsid)
        }
        
        #If ClinVar checkbox is selected, select those variants with at least one of the Clinical Significances chosen
        if (input$filterClinVarInput == T){
            separatedCV <- separate_rows(filtered, ClinicalSignificances, sep = ', ')
            ids_filter_CV <- separatedCV %>% filter(ClinicalSignificances %in% input$ClinVarInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_CV$rsid)
        }
        
        #If Regulome checkbox is selected, select those variants with a regulome rank score value
        #inside the chosen range
        if (input$filterRegulomeInput == T){
            filtered <- filtered %>% filter(RegulomeImportance >= match(input$RegulomeInput[1], regulomeLabels),
                                            RegulomeImportance <= match(input$RegulomeInput[2], regulomeLabels))
        }
        
        #If DSI checbox is selected, select those variants with a DSI value inside the chosen range
        if (input$filterDSIInput == T){
            filtered <- filtered %>% filter (`max(DSI)` >= input$DSIInput[1],
                                             `max(DSI)` <= input$DSIInput[2])
        }
        
        #If DPI checbox is selected, select those variants with a DPI value inside the chosen range
        if (input$filterDPIInput == T){
            filtered <- filtered %>% filter (`max(DPI)` >= input$DPIInput[1],
                                             `max(DPI)` <= input$DPIInput[2])
        }
        
        #If score checbox is selected, select those variants with a DisGeNET score inside the chosen range
        if (input$filterScoreInput == T){
            filtered <- filtered %>% filter (`max(score)` >= input$DGNscoreInput[1],
                                             `max(score)` <= input$DGNscoreInput[2])
        }
        
        #If N.of PubMed IDs checbox is selected, select those variants with a number of 
        #PubMed IDs value inside the chosen range
        if (input$filterNofPmidsInput == T){
            filtered <- filtered %>% filter (`max(NofPmids)` >= input$NofPmidsInput[1],
                                             `max(NofPmids)` <= input$NofPmidsInput[2])
        }
        
        #If Age filter checbox is selected, select those variants with a Age mean and score value, 
        #of the chosen clock model, inside the chosen ranges
        if (input$filterAgeInput == T){
            if ('Mut' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Mut >= input$AgeMeanInput[1],
                                                AgeMean_Mut <= input$AgeMeanInput[2],
                                                QualScore_Mut >= input$qualScoreInput[1],
                                                QualScore_Mut <= input$qualScoreInput[2])
            }
            if ('Rec' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Rec >= input$AgeMeanInput[1],
                                                AgeMean_Rec <= input$AgeMeanInput[2],
                                                QualScore_Rec >= input$qualScoreInput[1],
                                                QualScore_Rec <= input$qualScoreInput[2])
            }
            if ('Jnt' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Jnt >= input$AgeMeanInput[1],
                                                AgeMean_Jnt <= input$AgeMeanInput[2],
                                                QualScore_Jnt >= input$qualScoreInput[1],
                                                QualScore_Jnt <= input$qualScoreInput[2])
            }
        }
        
        #Design the plot
        o <- ggplot(filtered, aes(x=AgeMean_Rec, y=..count.., fill = ..count..)) + geom_histogram() 
        o + scale_fill_gradient(low='skyblue', high = 'purple')
        
    })
    
    #AGE MEAN (JOINT CLOCK) PLOT
    output$plotAgeMeanJnt <- renderPlot({
        
        #Filter the original data by position and save it in filtered
        filtered <- dades %>% filter(chr %in% input$chrInput,
                                     physicalPos >= input$minPosInput,
                                     physicalPos <= input$maxPosInput)
        
        #if extreme values checkbox is selected, select those variants that have 
        #an extreme value in at least one of the populations selected
        if (input$extremeValuesInput == TRUE){
            for (POP in pops){
                if (POP %in% input$popsInput){
                    llista_rsid <- c(llista_rsid, 
                                     filtered %>% filter(!is.na(filtered[[paste0('extremevalue_',POP)]])) %>%
                                         select(rsid))
                        
                    }
            }
                filtered <- filtered %>% filter(rsid %in% llista_rsid)      
        }
        
        #If GWAS checkbox is selected, select those variants that have a GWAS Association Count 
        #value inside the chosen range 
        if (input$filterGWASInput == T){
            filtered <- filtered %>% filter(GWASAssocCounts >= input$GWASInput[1],
                                            GWASAssocCounts <= input$GWASInput[2])
        }
        
        #If VEP checkbox is selected, select those variants with at least one of the VEP Consequences chosen
        if (input$filterVEPInput == T){
            separatedVEP <- separate_rows(filtered, VEPConseqs, sep = ',')
            ids_filter_VEP <- separatedVEP %>% filter(VEPConseqs %in% input$VEPInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_VEP$rsid)
        }
        
        #If ClinVar checkbox is selected, select those variants with at least one of the Clinical Significances chosen
        if (input$filterClinVarInput == T){
            separatedCV <- separate_rows(filtered, ClinicalSignificances, sep = ', ')
            ids_filter_CV <- separatedCV %>% filter(ClinicalSignificances %in% input$ClinVarInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_CV$rsid)
        }
        
        #If Regulome checkbox is selected, select those variants with a regulome rank score value
        #inside the chosen range
        if (input$filterRegulomeInput == T){
            filtered <- filtered %>% filter(RegulomeImportance >= match(input$RegulomeInput[1], regulomeLabels),
                                            RegulomeImportance <= match(input$RegulomeInput[2], regulomeLabels))
        }
        
        #If DSI checbox is selected, select those variants with a DSI value inside the chosen range
        if (input$filterDSIInput == T){
            filtered <- filtered %>% filter (`max(DSI)` >= input$DSIInput[1],
                                             `max(DSI)` <= input$DSIInput[2])
        }
        
        #If DPI checbox is selected, select those variants with a DPI value inside the chosen range
        if (input$filterDPIInput == T){
            filtered <- filtered %>% filter (`max(DPI)` >= input$DPIInput[1],
                                             `max(DPI)` <= input$DPIInput[2])
        }
        
        #If score checbox is selected, select those variants with a DisGeNET score inside the chosen range
        if (input$filterScoreInput == T){
            filtered <- filtered %>% filter (`max(score)` >= input$DGNscoreInput[1],
                                             `max(score)` <= input$DGNscoreInput[2])
        }
        
        #If N.of PubMed IDs checbox is selected, select those variants with a number of 
        #PubMed IDs value inside the chosen range
        if (input$filterNofPmidsInput == T){
            filtered <- filtered %>% filter (`max(NofPmids)` >= input$NofPmidsInput[1],
                                             `max(NofPmids)` <= input$NofPmidsInput[2])
        }
        
        #If Age filter checbox is selected, select those variants with a Age mean and score value, 
        #of the chosen clock model, inside the chosen ranges
        if (input$filterAgeInput == T){
            if ('Mut' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Mut >= input$AgeMeanInput[1],
                                                AgeMean_Mut <= input$AgeMeanInput[2],
                                                QualScore_Mut >= input$qualScoreInput[1],
                                                QualScore_Mut <= input$qualScoreInput[2])
            }
            if ('Rec' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Rec >= input$AgeMeanInput[1],
                                                AgeMean_Rec <= input$AgeMeanInput[2],
                                                QualScore_Rec >= input$qualScoreInput[1],
                                                QualScore_Rec <= input$qualScoreInput[2])
            }
            if ('Jnt' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Jnt >= input$AgeMeanInput[1],
                                                AgeMean_Jnt <= input$AgeMeanInput[2],
                                                QualScore_Jnt >= input$qualScoreInput[1],
                                                QualScore_Jnt <= input$qualScoreInput[2])
            }
        }
        
        #Design the plot
        o <- ggplot(filtered, aes(x=AgeMean_Jnt, y=..count.., fill = ..count..)) + geom_histogram() 
        o + scale_fill_gradient(low='skyblue', high = 'purple')
        
    })
        
        #RESULTS TABLE
        output$tableResults <-  renderDataTable({
            
        #Filter the original data by position and save it in filtered
        filtered <- dades %>% filter(chr %in% input$chrInput,
                                     physicalPos >= input$minPosInput,
                                     physicalPos <= input$maxPosInput)
        
        #if extreme values checkbox is selected, select those variants that have 
        #an extreme value in at least one of the populations selected
        if (input$extremeValuesInput == TRUE){
            for (POP in pops){
                if (POP %in% input$popsInput){
                    llista_rsid <- c(llista_rsid, 
                                     filtered %>% filter(!is.na(filtered[[paste0('extremevalue_',POP)]])) %>%
                                         select(rsid))
                        
                    }
            }
                filtered <- filtered %>% filter(rsid %in% llista_rsid)      
        }
        
        #If GWAS checkbox is selected, select those variants that have a GWAS Association Count 
        #value inside the chosen range 
        if (input$filterGWASInput == T){
            filtered <- filtered %>% filter(GWASAssocCounts >= input$GWASInput[1],
                                            GWASAssocCounts <= input$GWASInput[2])
        }
        
        #If VEP checkbox is selected, select those variants with at least one of the VEP Consequences chosen
        if (input$filterVEPInput == T){
            separatedVEP <- separate_rows(filtered, VEPConseqs, sep = ',')
            ids_filter_VEP <- separatedVEP %>% filter(VEPConseqs %in% input$VEPInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_VEP$rsid)
        }
        
        #If ClinVar checkbox is selected, select those variants with at least one of the Clinical Significances chosen
        if (input$filterClinVarInput == T){
            separatedCV <- separate_rows(filtered, ClinicalSignificances, sep = ', ')
            ids_filter_CV <- separatedCV %>% filter(ClinicalSignificances %in% input$ClinVarInput)
            filtered <- filtered %>% filter (rsid %in% ids_filter_CV$rsid)
        }
        
        #If Regulome checkbox is selected, select those variants with a regulome rank score value
        #inside the chosen range
        if (input$filterRegulomeInput == T){
            filtered <- filtered %>% filter(RegulomeImportance >= match(input$RegulomeInput[1], regulomeLabels),
                                            RegulomeImportance <= match(input$RegulomeInput[2], regulomeLabels))
        }
        
        #If DSI checbox is selected, select those variants with a DSI value inside the chosen range
        if (input$filterDSIInput == T){
            filtered <- filtered %>% filter (`max(DSI)` >= input$DSIInput[1],
                                             `max(DSI)` <= input$DSIInput[2])
        }
        
        #If DPI checbox is selected, select those variants with a DPI value inside the chosen range
        if (input$filterDPIInput == T){
            filtered <- filtered %>% filter (`max(DPI)` >= input$DPIInput[1],
                                             `max(DPI)` <= input$DPIInput[2])
        }
        
        #If score checbox is selected, select those variants with a DisGeNET score inside the chosen range
        if (input$filterScoreInput == T){
            filtered <- filtered %>% filter (`max(score)` >= input$DGNscoreInput[1],
                                             `max(score)` <= input$DGNscoreInput[2])
        }
        
        #If N.of PubMed IDs checbox is selected, select those variants with a number of 
        #PubMed IDs value inside the chosen range
        if (input$filterNofPmidsInput == T){
            filtered <- filtered %>% filter (`max(NofPmids)` >= input$NofPmidsInput[1],
                                             `max(NofPmids)` <= input$NofPmidsInput[2])
        }
        
        #If Age filter checbox is selected, select those variants with a Age mean and score value, 
        #of the chosen clock model, inside the chosen ranges
        if (input$filterAgeInput == T){
            if ('Mut' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Mut >= input$AgeMeanInput[1],
                                                AgeMean_Mut <= input$AgeMeanInput[2],
                                                QualScore_Mut >= input$qualScoreInput[1],
                                                QualScore_Mut <= input$qualScoreInput[2])
            }
            if ('Rec' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Rec >= input$AgeMeanInput[1],
                                                AgeMean_Rec <= input$AgeMeanInput[2],
                                                QualScore_Rec >= input$qualScoreInput[1],
                                                QualScore_Rec <= input$qualScoreInput[2])
            }
            if ('Jnt' %in% input$AgeModelInput){
                filtered <- filtered %>% filter(AgeMean_Jnt >= input$AgeMeanInput[1],
                                                AgeMean_Jnt <= input$AgeMeanInput[2],
                                                QualScore_Jnt >= input$qualScoreInput[1],
                                                QualScore_Jnt <= input$qualScoreInput[2])
            }
        }
            #Don't select the columns made by us (numeric impacts)
            return(filtered %>% select(-MaxNumConseq, -BinaryConseq, -ClinicalImpacts, -MaxClinicalImpact, -RegulomeImportance)) 
                  
        })
    
#DOWNLOAD BUTTON <- not working yet
        thedata <- reactive({
            filtered <- dades %>% filter(chr %in% input$chrInput,
                                         physicalPos >= input$minPosInput,
                                         physicalPos <= input$maxPosInput) 
            
            if (input$extremeValuesInput == TRUE){
                for (POP in pops){
                    if (POP %in% input$popsInput){
                        llista_rsid <- c(llista_rsid, 
                                         filtered %>% filter(!is.na(filtered[[paste0('extremevalue_',POP)]])) %>%
                                             select(rsid))
                        
                    }
                }
                filtered <- filtered %>% filter(rsid %in% llista_rsid)  
                
                
            }
            if (input$filterGWASInput == T){
                filtered <- filtered %>% filter(GWASAssocCounts >= input$GWASInput[1],
                                                GWASAssocCounts <= input$GWASInput[2])
            }
            if (input$filterVEPInput == T){
                separatedVEP <- separate_rows(filtered, VEPConseqs, sep = ',')
                ids_filter_VEP <- separatedVEP %>% filter(VEPConseqs %in% input$VEPInput)
                filtered <- filtered %>% filter (rsid %in% ids_filter_VEP$rsid)
            }
            if (input$filterClinVarInput == T){
                separatedCV <- separate_rows(filtered, ClinicalSignificances, sep = ', ')
                ids_filter_CV <- separatedCV %>% filter(ClinicalSignificances %in% input$ClinVarInput)
                filtered <- filtered %>% filter (rsid %in% ids_filter_CV$rsid)
            }
            if (input$filterRegulomeInput == T){
                filtered <- filtered %>% filter(RegulomeImportance >= match(input$RegulomeInput[1], regulomeLabels),
                                                RegulomeImportance <= match(input$RegulomeInput[2], regulomeLabels))
            }
            if (input$filterDSIInput == T){
                filtered <- filtered %>% filter (`max(DSI)` >= input$DSIInput[1],
                                                 `max(DSI)` <= input$DSIInput[2])
            }
            if (input$filterDPIInput == T){
                filtered <- filtered %>% filter (`max(DPI)` >= input$DPIInput[1],
                                                 `max(DPI)` <= input$DPIInput[2])
            }
            if (input$filterScoreInput == T){
                filtered <- filtered %>% filter (`max(score)` >= input$DGNscoreInput[1],
                                                 `max(score)` <= input$DGNscoreInput[2])
            }
            if (input$filterNofPmidsInput == T){
                filtered <- filtered %>% filter (`max(NofPmids)` >= input$NofPmidsInput[1],
                                                 `max(NofPmids)` <= input$NofPmidsInput[2])
            }
            if (input$filterAgeInput == T){
                if ('Mut' %in% input$AgeModelInput){
                    filtered <- filtered %>% filter(AgeMean_Mut >= input$AgeMeanInput[1],
                                                    AgeMean_Mut <= input$AgeMeanInput[2],
                                                    QualScore_Mut >= input$qualScoreInput[1],
                                                    QualScore_Mut <= input$qualScoreInput[2])
                }
                if ('Rec' %in% input$AgeModelInput){
                    filtered <- filtered %>% filter(AgeMean_Rec >= input$AgeMeanInput[1],
                                                    AgeMean_Rec <= input$AgeMeanInput[2],
                                                    QualScore_Rec >= input$qualScoreInput[1],
                                                    QualScore_Rec <= input$qualScoreInput[2])
                }
                if ('Jnt' %in% input$AgeModelInput){
                    filtered <- filtered %>% filter(AgeMean_Jnt >= input$AgeMeanInput[1],
                                                    AgeMean_Jnt <= input$AgeMeanInput[2],
                                                    QualScore_Jnt >= input$qualScoreInput[1],
                                                    QualScore_Jnt <= input$qualScoreInput[2])
                }
            }
            return(filtered)
        })
        output$download <- downloadHandler(
            filename = function(){'Data.cvs'},
            content = function(fname){
                write.csv(thedata(), fname)
            } #es baixara l'arxiu amb el nom de DATA
        )
}


# Run the application 
shinyApp(ui = ui, server = server)
